# Doc2Vec

## 0. Data Load

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
! bash install_mecab-ko_on_colab190912.sh

/content
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.7MB/s 
     |████████████████████████████████| 450kB 30.6MB/s 
     |████████████████████████████████| 92kB 1.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-07-17 05:32:20--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::34cc:ea4a, 2406:da00:ff00::22c3:9b0a

In [3]:
from konlpy.tag import Mecab
mecab = Mecab()

In [4]:
import pandas as pd
import numpy as np
import nltk, re
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
import os
os.chdir('/content/drive/MyDrive/BOAZ_ADV_대법관_김보아즈/data')

In [6]:
news = pd.read_csv('뉴스/실험군/news_for_sim.csv')
laws = pd.read_csv('뉴스/실험군/0711law.csv')

In [ ]:
news.head(2)

,제목,본문,키워드,카테고리
0,"통일부 논쟁 종막? 이인영 ""기꺼이 져드릴게"" 이준석 ""민주주의 인권이나 지지 말길""",이준석 국민의힘 대표와 이틀 간 통일부 존폐를 놓고 설전(舌戰)을 벌인 이인영 통일...,"종막,통일부,논쟁,이인영,이준석,민주주의,인권,이인영,직성,애초,북한,인권,얘기,통...",살인
1,"서울대 학생처장 “산 사람들, 피해자 코스프레 역겹다"" 청소노동자 갑질 의혹 반박",지난 7일 서울대학교에서 ‘서울대학교 청소 노동자 조합원 사망 관련 서울대학교 오세...,"서울대,학생처장,사람,피해자,코스프레,청소,노동자,갑질,의혹,반박,서울대학교,사망,...",절도


In [ ]:
laws.head(2)

,Unnamed: 0,category,article_original,abstract,kobart_sum
0,0,else,검사의 상고이유를 본다.원심은 피고인들에 대한 공정증서 원본 불실기재 동행사의 ...,"외국판결에 의하여 신고한 경우 공정증서원본 분실기재, 동 행사죄의 성립을 부정한 사례",피고인들에게들에 대한 공정증서 원본 불실기재 동행사의 점에 관하여는 피고인들이 미국...
1,1,else,"피고소송대리인들의 상고이유 제1, 2점에 대하여,원판결이 든 증거에 의하여 원판...",대물변제 예약의 법리를 오해하여 부동산의 인도지연으로 인한 입료상당의 손해금의 지...,상고이유 제3점에 대하여 원판결이 든 증거에 의하여 원판결의 소론 판단 사실을 수긍...


## 1. Data Preprocessing

In [ ]:
len(laws[laws['kobart_sum'].isnull()])

37040

In [7]:
# NA 제거
laws.drop(['Unnamed: 0'], axis=1, inplace=True)
laws_na = laws[laws['kobart_sum'].notnull()].reset_index(drop=True)

In [8]:
# 전처리
def text_cleaning(texts):
    string = re.sub(r'[·@%\\*=()/~#&\+á‘’“”?\xc3\xa1\-\|\:\;\!\-\,\_\~\$\'\"\[\]]', ' ', texts) #remove punctuation
    string = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ", string)
    string = re.sub(r'\n',' ', string)    # remove enter
    string = re.sub(r'[0-9]+', '', string)
    string = re.sub(r'\s+', ' ', string)  #remove extra space
    string = re.sub(r'<[^>]+>',' ',string) #remove Html tags
    string = re.sub(r'www.jeju.go.kr',' ',string) #remove url
    string = re.sub(r'jmp',' ',string) #remove url
    string = re.sub(r'index.htm',' ',string) #remove url
    return string

In [9]:
# 불용어 정의
sw = set(['항소', '법원', '법', '재판'])

In [10]:
# 토큰화
def text_tokenizing(doc,tokenizer):
  if tokenizer == "words":
    return [word for word in doc.split() if word not in sw]
  elif tokenizer == "nouns":
    return [word for word in mecab.nouns(doc) if word not in sw]
  elif tokenizer == "morphs":
    return [word for word in mecab.morphs(doc) if word not in sw]
  return words

In [11]:
# 전처리
laws_na['prepro'] = laws_na['kobart_sum'].progress_apply(text_cleaning)
# 토큰화
laws_data = laws_na['prepro'].tolist()
laws_na['nouns'] = [text_tokenizing(law, tokenizer = "nouns") for law in tqdm(laws_data)]

100%|██████████| 40342/40342 [00:54<00:00, 746.03it/s]


In [12]:
laws_na.head(2)

,category,article_original,abstract,kobart_sum,prepro,nouns
0,else,검사의 상고이유를 본다.원심은 피고인들에 대한 공정증서 원본 불실기재 동행사의 ...,"외국판결에 의하여 신고한 경우 공정증서원본 분실기재, 동 행사죄의 성립을 부정한 사례",피고인들에게들에 대한 공정증서 원본 불실기재 동행사의 점에 관하여는 피고인들이 미국...,피고인들에게들에 대한 공정증서 원본 불실기재 동행사의 점에 관하여는 피고인들이 미국...,"[피고인, 들, 공정증서, 원본, 불실, 기재, 동행, 사, 점, 피고인, 미국, ..."
1,else,"피고소송대리인들의 상고이유 제1, 2점에 대하여,원판결이 든 증거에 의하여 원판...",대물변제 예약의 법리를 오해하여 부동산의 인도지연으로 인한 입료상당의 손해금의 지...,상고이유 제3점에 대하여 원판결이 든 증거에 의하여 원판결의 소론 판단 사실을 수긍...,상고이유 제 점에 대하여 원판결이 든 증거에 의하여 원판결의 소론 판단 사실을 수긍...,"[상고, 이유, 점, 원판결, 증거, 원판결, 소론, 판단, 사실, 수긍, 바, 검..."


In [13]:
news['noun'] = news['키워드'].progress_apply(lambda x: x.split(','))

100%|██████████| 46625/46625 [00:01<00:00, 29767.17it/s]


## 2. Category

In [ ]:
laws_na['category'].value_counts()

else    37249
폭행       1062
교통        805
절도        612
살인        275
성관련       221
마약        118
Name: category, dtype: int64

In [ ]:
news['카테고리'].value_counts()

폭행      21017
성관련      7148
마약       5700
살인       4358
노상사고     3483
절도       2612
사기       2307
Name: 카테고리, dtype: int64

In [15]:
laws_na = laws_na[laws_na['category'] != 'else'].reset_index(drop=True)

In [27]:
news['카테고리'] = news['카테고리'].progress_apply(lambda x: '교통' if x=='노상사고' else x)

100%|██████████| 46625/46625 [00:00<00:00, 523000.83it/s]


In [16]:
laws_na['category'].value_counts()

폭행     1062
교통      805
절도      612
살인      275
성관련     221
마약      118
Name: category, dtype: int64

In [28]:
news['카테고리'].value_counts()

폭행     21017
성관련     7148
마약      5700
살인      4358
교통      3483
절도      2612
사기      2307
Name: 카테고리, dtype: int64

## 3. Doc2Vec

### 1. Train

In [ ]:
laws_na.head(2)

,category,article_original,abstract,kobart_sum,prepro,nouns
0,else,검사의 상고이유를 본다.원심은 피고인들에 대한 공정증서 원본 불실기재 동행사의 ...,"외국판결에 의하여 신고한 경우 공정증서원본 분실기재, 동 행사죄의 성립을 부정한 사례",피고인들에게들에 대한 공정증서 원본 불실기재 동행사의 점에 관하여는 피고인들이 미국...,피고인들에게들에 대한 공정증서 원본 불실기재 동행사의 점에 관하여는 피고인들이 미국...,"[피고인, 들, 공정증서, 원본, 불실, 기재, 동행, 사, 점, 피고인, 미국, ..."
1,else,"피고소송대리인들의 상고이유 제1, 2점에 대하여,원판결이 든 증거에 의하여 원판...",대물변제 예약의 법리를 오해하여 부동산의 인도지연으로 인한 입료상당의 손해금의 지...,상고이유 제3점에 대하여 원판결이 든 증거에 의하여 원판결의 소론 판단 사실을 수긍...,상고이유 제 점에 대하여 원판결이 든 증거에 의하여 원판결의 소론 판단 사실을 수긍...,"[상고, 이유, 점, 원판결, 증거, 원판결, 소론, 판단, 사실, 수긍, 바, 검..."


In [29]:
law_doc = laws_na.reset_index()

In [30]:
doc_df = law_doc[['index','nouns']].values.tolist()
tagged_data = [TaggedDocument(words=words, tags=[id]) for id, words in doc_df]

In [47]:
max_epochs = 10

model = Doc2Vec(
    window = 10,
    size = 150,
    alpha = 0.025, 
    min_alpha = 0.025,
    min_count = 2,
    dm = 1,
    negative = 5,
    seed = 1234)
  
model.build_vocab(tagged_data)

for epoch in tqdm(range(max_epochs)):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 10/10 [00:37<00:00,  3.72s/it]


In [ ]:
news.head(2)

,제목,본문,키워드,카테고리,noun
0,"통일부 논쟁 종막? 이인영 ""기꺼이 져드릴게"" 이준석 ""민주주의 인권이나 지지 말길""",이준석 국민의힘 대표와 이틀 간 통일부 존폐를 놓고 설전(舌戰)을 벌인 이인영 통일...,"종막,통일부,논쟁,이인영,이준석,민주주의,인권,이인영,직성,애초,북한,인권,얘기,통...",살인,"[종막, 통일부, 논쟁, 이인영, 이준석, 민주주의, 인권, 이인영, 직성, 애초,..."
1,"서울대 학생처장 “산 사람들, 피해자 코스프레 역겹다"" 청소노동자 갑질 의혹 반박",지난 7일 서울대학교에서 ‘서울대학교 청소 노동자 조합원 사망 관련 서울대학교 오세...,"서울대,학생처장,사람,피해자,코스프레,청소,노동자,갑질,의혹,반박,서울대학교,사망,...",절도,"[서울대, 학생처장, 사람, 피해자, 코스프레, 청소, 노동자, 갑질, 의혹, 반박..."


### 2. Top1

* else 포함

In [38]:
model.random.seed(1234)
news['sim_cat'] = ''
news['sim_law'] = ''
news['similarity'] = ''

for i in tqdm(range(len(news))):
    doc_list = news['noun'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=1)
    news['sim_cat'][i] = law_doc['category'][return_docs[0][0]]
    news['sim_law'][i] = law_doc['kobart_sum'][return_docs[0][0]]
    news['similarity'][i] = return_docs[0][1]

  0%|          | 0/46625 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [39]:
news.head(2)

,제목,본문,키워드,카테고리,noun,sim_cat,similarity,acc,sim_law
0,"통일부 논쟁 종막? 이인영 ""기꺼이 져드릴게"" 이준석 ""민주주의 인권이나 지지 말길""",이준석 국민의힘 대표와 이틀 간 통일부 존폐를 놓고 설전(舌戰)을 벌인 이인영 통일...,"종막,통일부,논쟁,이인영,이준석,민주주의,인권,이인영,직성,애초,북한,인권,얘기,통...",살인,"[종막, 통일부, 논쟁, 이인영, 이준석, 민주주의, 인권, 이인영, 직성, 애초,...",절도,0.0464121,False,선고일자 기재는 오기임이 명백하므로 직권으로 주문과 같이 결정한다.
1,"서울대 학생처장 “산 사람들, 피해자 코스프레 역겹다"" 청소노동자 갑질 의혹 반박",지난 7일 서울대학교에서 ‘서울대학교 청소 노동자 조합원 사망 관련 서울대학교 오세...,"서울대,학생처장,사람,피해자,코스프레,청소,노동자,갑질,의혹,반박,서울대학교,사망,...",절도,"[서울대, 학생처장, 사람, 피해자, 코스프레, 청소, 노동자, 갑질, 의혹, 반박...",폭행,0.150182,False,택시운행방해죄와 공동폭행죄는 서로 다른 죄가 성립함에도 피고인들이 택시운행을 방해할...


In [ ]:
# 코사인 유사도
np.mean(news['similarity'])

0.2782697181980707

In [ ]:
# 정확도
news['acc'] = [news['카테고리'][i] == news['sim_cat'][i] for i in range(len(news))]
sum(news['acc'])/len(news)

0.058680965147453086

* else 제거

In [48]:
model.random.seed(1234)
news['sim_cat'] = ''
news['similarity'] = ''

for i in tqdm(range(len(news))):
    doc_list = news['noun'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=1)
    news['sim_cat'][i] = law_doc['category'][return_docs[0][0]]
    news['similarity'][i] = return_docs[0][1]

  0%|          | 0/46625 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 46625/46625 [03:39<00:00, 212.59it/s]


In [49]:
# 코사인 유사도
np.mean(news['similarity'])

0.16727016047540122

In [50]:
# 정확도
news['acc'] = [news['카테고리'][i] == news['sim_cat'][i] for i in range(len(news))]
sum(news['acc'])/len(news)

0.35710455764075066

In [51]:
news.head(2)

,제목,본문,키워드,카테고리,noun,sim_cat,similarity,acc,sim_law
0,"통일부 논쟁 종막? 이인영 ""기꺼이 져드릴게"" 이준석 ""민주주의 인권이나 지지 말길""",이준석 국민의힘 대표와 이틀 간 통일부 존폐를 놓고 설전(舌戰)을 벌인 이인영 통일...,"종막,통일부,논쟁,이인영,이준석,민주주의,인권,이인영,직성,애초,북한,인권,얘기,통...",살인,"[종막, 통일부, 논쟁, 이인영, 이준석, 민주주의, 인권, 이인영, 직성, 애초,...",절도,0.0556756,False,선고일자 기재는 오기임이 명백하므로 직권으로 주문과 같이 결정한다.
1,"서울대 학생처장 “산 사람들, 피해자 코스프레 역겹다"" 청소노동자 갑질 의혹 반박",지난 7일 서울대학교에서 ‘서울대학교 청소 노동자 조합원 사망 관련 서울대학교 오세...,"서울대,학생처장,사람,피해자,코스프레,청소,노동자,갑질,의혹,반박,서울대학교,사망,...",절도,"[서울대, 학생처장, 사람, 피해자, 코스프레, 청소, 노동자, 갑질, 의혹, 반박...",폭행,0.234442,False,택시운행방해죄와 공동폭행죄는 서로 다른 죄가 성립함에도 피고인들이 택시운행을 방해할...


In [52]:
news.to_csv('/content/drive/MyDrive/BOAZ_ADV_대법관_김보아즈/data/뉴스/실험군/0717_doc2Vec.csv', index=False)

* epoch

In [42]:
model.random.seed(1234)
news['sim_cat'] = ''
news['similarity'] = ''

for i in tqdm(range(len(news))):
    doc_list = news['noun'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=1)
    news['sim_cat'][i] = law_doc['category'][return_docs[0][0]]
    news['similarity'][i] = return_docs[0][1]

  0%|          | 0/46625 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 46625/46625 [03:25<00:00, 226.36it/s]


In [43]:
# 코사인 유사도
np.mean(news['similarity'])

0.5244338147825593

In [44]:
# 정확도
news['acc'] = [news['카테고리'][i] == news['sim_cat'][i] for i in range(len(news))]
sum(news['acc'])/len(news)

0.18979088471849867

### 3. Top5

* else 포함

In [ ]:
model.random.seed(1234)
accuracy = 0

for i in tqdm(range(len(news))):
    doc_list = news['noun'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=5)
    sim_cat = [law_doc['category'][return_docs[0][0]], law_doc['category'][return_docs[1][0]], law_doc['category'][return_docs[2][0]], law_doc['category'][return_docs[3][0]], law_doc['category'][return_docs[4][0]]]
    if news['카테고리'][i] in sim_cat: accuracy += 1

100%|██████████| 46625/46625 [13:15<00:00, 58.62it/s]


In [ ]:
# 정확도
accuracy/len(news)

0.25083109919571045

* else 제거

In [54]:
model.random.seed(1234)
accuracy = 0

for i in tqdm(range(len(news))):
    doc_list = news['noun'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=5)
    sim_cat = [law_doc['category'][return_docs[0][0]], law_doc['category'][return_docs[1][0]], law_doc['category'][return_docs[2][0]], law_doc['category'][return_docs[3][0]], law_doc['category'][return_docs[4][0]]]
    if news['카테고리'][i] in sim_cat: accuracy += 1

100%|██████████| 46625/46625 [02:51<00:00, 271.16it/s]


In [55]:
# 정확도
accuracy/len(news)

0.7942949061662199

* epoch

In [45]:
model.random.seed(1234)
accuracy = 0

for i in tqdm(range(len(news))):
    doc_list = news['noun'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=5)
    sim_cat = [law_doc['category'][return_docs[0][0]], law_doc['category'][return_docs[1][0]], law_doc['category'][return_docs[2][0]], law_doc['category'][return_docs[3][0]], law_doc['category'][return_docs[4][0]]]
    if news['카테고리'][i] in sim_cat: accuracy += 1

100%|██████████| 46625/46625 [02:38<00:00, 294.11it/s]


In [46]:
# 정확도
accuracy/len(news)

0.5780160857908847